In [ ]:
import pandas as pd                               # panda
import pandas_datareader as pdr                   # Baixar dados yahoo

import matplotlib.pyplot as plt                   # grafico
import matplotlib.dates as mdates                 # trabalhar datas

import statsmodels.api as sm                      # regressao
from statsmodels.api import add_constant          # para a regressao considerar o intercepto

import math as math
import numpy as np
import scipy.stats


from statsmodels.tsa.stattools import coint       # Funcao Coint ()
from statsmodels.tsa.stattools import adfuller

# Declaracoes
pd.options.display.float_format = '{:.2f}'.format

#!pip install MetaTrader5
#!pip install --upgrade MetaTrader5
import MetaTrader5 as mt5
from datetime import datetime
from datetime import date
from datetime import timedelta
import time
import pytz
import sys

import ipywidgets

%matplotlib inline

In [ ]:
def Get_data(tickers,inicio,fim,janela):

    # Checa se ja estamos conectados, se não, conecta
    if (mt5.terminal_info()==None):
          
        # conecte-se ao MetaTrader 5
        if not mt5.initialize():
            print("initialize() failed, error code =",mt5.last_error())
            mt5.shutdown()
        
    #quebra as strings dates(inicio,fim) recebidas e transformando em integer
    inicio_= inicio.split('-')
    inicio = [int(element) for element in inicio_]
    fim_= fim.split('-')
    fim = [int(element) for element in fim_]

    timezone = pytz.timezone("Etc/UTC")
    
    #Verifica caso seja timeframe diario ou intraday
    if janela == '1d':
        dt_inicio = datetime(inicio[0], inicio[1], inicio[2], tzinfo=timezone)
        dt_fim = datetime(fim[0], fim[1], fim[2], tzinfo=timezone)
        timeframe = mt5.TIMEFRAME_D1
    #aqui trata as datas intraday
    else:
        dt_inicio = datetime(inicio[0], inicio[1], inicio[2], inicio[3], inicio[4], tzinfo=timezone)
        dt_fim = datetime(fim[0], fim[1], fim[2], fim[3], fim[4], tzinfo=timezone)
        timeframe = mt5.TIMEFRAME_M15
        
    # obtendo o par de acoes com base nas datas enviadas nas strings, caso len seja 1, getdata retorna o dataframe de uma unica acao
    if (len (tickers) == 1):        
        stock1 = mt5.copy_rates_range(tickers[0], timeframe, dt_inicio, dt_fim)    
        stock_df = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
                
    else:    
        stock1 = mt5.copy_rates_range(tickers[0], timeframe, dt_inicio, dt_fim)
        stock2 = mt5.copy_rates_range(tickers[1], timeframe, dt_inicio, dt_fim)
        esquerda = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
        direita = pd.DataFrame(stock2)[['close','time']].rename(columns={'close': tickers[1]})
        stock_df = pd.merge(left=esquerda, right=direita, left_on='time', right_on='time')
    
    #Se o timeframe for intraday, retorna um datetime (com minutos e horas), caso contrario, retorna um date
    if (janela != '1d'):
        stock_df['Date'] = stock_df['time'].apply(lambda x: str (datetime.fromtimestamp(x, tz=timezone))[:-9]) #define timezone
    else:
        stock_df['Date'] = stock_df['time'].apply(lambda x: date.fromtimestamp(x+21600)) #adicionando 21600 pra corrigir o fuso no momento de converter tsp em str
        stock_df.drop('time',axis=1,inplace = True)
        
    return stock_df.set_index('Date')

In [ ]:
# Gera a REGRESSAO LINEAR MULTIVARIADA (QuantGo "Simples") -> y = b*x + c*t + d + e
def regressao_multivariada(x, y, periodo):  
    
    X = np.column_stack((x, range(1,periodo+1,1)))
    X = sm.add_constant(X)
    res = sm.OLS(y,X).fit()
    coef_ang=res.params[1]
    residuo = y-res.predict()
    zscore  = (residuo - np.mean(residuo))/np.std(residuo);
    return coef_ang, residuo, zscore, res


def regressao_residuos(residuos):
  
    residuos_shifted = residuos.diff(1).fillna(method="bfill")  
    delta = residuos - residuos_shifted
    X = sm.add_constant(residuos_shifted, prepend=True)
    res = sm.OLS(residuos,X).fit()
    return res.params[1]

# ADF da Statsmodel e teste de significancia com os critical values do Ferro
def adftest(df,reg):

    adf  = adfuller(df,maxlag=1, autolag="BIC")
    
    adf_stat = adf[0]
    adf_pvalue = adf[1]
    
    ## %ADF clássico, de acordo com o output da função  - NAO UTILIZADO
    if   adf[0] < adf[4]['1%']:  adfc='99%'
    elif adf[0] < adf[4]['5%']:  adfc='95%'
    elif adf[0] < adf[4]['10%']: adfc='90%'
    else:                        adfc='-'                 

    ## %ADF do Ferro
    # Utiliza duas tabelas - uma para quando o tempo é significante, outra quando não é (indep. do noobs)
    ttest = reg.params[1]/reg.bse[1]
    critical_value_tempo = scipy.stats.t.ppf(0.01,df=(len(df)-3)) # Retorna o inverso bicaudal da distribuição t de Student

    if (abs(ttest) < critical_value_tempo): aceitar_t0 = '99%'
    else:  aceitar_t0 = '0%'
    
    if (aceitar_t0 == '0%'):
        if   adf[0] < -4.32:         adf_sign='99%'
        elif adf[0] < -3.67:         adf_sign='95%'
        elif adf[0] < -3.28:         adf_sign='90%'
        else:                        adf_sign='-' 
    else:    
        if   adf[0] < -3.58:         adf_sign='99%'
        elif adf[0] < -3.22:         adf_sign='95%'
        elif adf[0] < -2.60:         adf_sign='90%'
        else:                        adf_sign='-'                 

    
    
        
    return adf_stat, adf_sign, adf_pvalue  


def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''
    
    r, p = scipy.stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = scipy.stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    
    return r, p, lo, hi
    
def pct_financeiro(x,y,coef_ang,residuo):
    
    ultimo_x = float(x.tail(1))
    ultimo_y = float(y.tail(1))
    
    fin_x = ultimo_x*coef_ang
    fin_y = ultimo_y
    
    if (residuo > 0):
        compra = fin_x
        venda = fin_y
    else:
        compra = fin_y
        venda = fin_x
 
    cv = "{:.0%}".format(compra/venda)
    return cv


def calculo_meia_vida1(residuo):
    # Retirado de um paper e adaptado pela formula da planilha do Ferro
    price = pd.Series(residuo)  
    lagged_price = price.shift(1).fillna(method="bfill")  
    delta = price - lagged_price  
    beta = np.polyfit(lagged_price, delta, 1)[0] 
    #half_life = ((-2*np.log(2))/beta)  # paper (varios), mas sem o 2*
    half_life = 2/(-1*np.log(1+beta))   # planilha ferro
     
    return (int(round(half_life)))


In [ ]:
def ajusta_B3Date (data_start, qtde_dias):
    
    df = pd.read_excel('Calendar.xlsx')
    df['Date'] = df['Date'].apply(str)
    try:
        index = df [df['Date'] == data_start].index.values.astype(int)[0]
        data_fim = df.iloc[index+qtde_dias][1]
    except:
        print ('****************************************************************')
        print ('ERRO, TENTATIVA DE ACESSO DE DATAS NAO INCLUIDAS EM CALENDAR.XLS')
        print ('****************************************************************\n')
        sys.exit()
    else:
        return (data_fim + '-17-00')

    
def simula_trade (ticker, beta_inicio, dp_entrada, dp_target, start_time , hlife):
    
    #passando a data de inicio do trade para identificarmos qual a data limite para o trade com base no calendario da B3
    end_time = ajusta_B3Date(start_time[:-6],hlife)
    #print (start_time)
    #print (end_time)
    stock = Get_data(ticker, start_time, end_time,'M5')
    
    return stock
    
def merge_intraday (ticker, dt_inicio, dt_intraday):

    dt_intraday_ant = datetime.strptime(dt_intraday, '%Y-%m-%d-%H-%M') 
    dt_intraday_ant = dt_intraday_ant - timedelta(days=1)  # Converte p/ data e subtrai 1 dia
    dt_fim = dt_intraday_ant.strftime('%Y-%m-%d-%H-%M')
    
    stock  = Get_data(ticker, dt_inicio,dt_fim,'1d')
    stock_intraday = Get_data(ticker, dt_intraday,dt_intraday,'M15')
    
    stock_intraday['Date'] =  stock_intraday['time'].apply(date.fromtimestamp)
    stock_intraday.set_index('Date', inplace = True)
    stock_concat = pd.concat([stock,stock_intraday])
    stock_concat.drop('time',axis=1,inplace = True)

    return stock_concat

In [ ]:
def Get_Cubo (tickers, dt_cubo, intra_bt):
    
    #Pegando a data B3 de 250 regoes atras
    if intra_bt:
        dt_inicio = ajusta_B3Date(dt_cubo[:-6],-250)
    else:
        dt_inicio = ajusta_B3Date(dt_cubo,-250)
    
    #print (dt_inicio)
    
    if intra_bt:
        #Para retornar o par com dados diarios, com ultimo dia sendo o intraday passado em dt_final
        par = merge_intraday (tickers, dt_inicio, dt_cubo)
    else:   
        # Busca cotacoes no Yahoo Finance
        par = Get_data(tickers, dt_inicio, dt_cubo,'1d')

    # Cria o cubo de periodos
    cubo = pd.DataFrame(columns=['preco_y','preco_x','ratio','periodo', 'adf_stat','adf_sign','coef_ang',
                                 'desvio','pct_fin','meia_vida', 'corr', 'fx_corr','per_coint','vol_beta_vertical','fx_vol_beta_vertical',
                                 'vol_beta_vertical1','fx_vol_beta_vertical1','adf99_periodos','dp_residuo'])

    # Popula o cubo de periodos
    #for i in range(100,260,10):
    for i in [100,120,140,160,180,200,220,240,250]:

        # Cria subset do tamanho do periodo atual da iteração e seta a variavel indep (x) e dependente (y)
        par_subset = par.tail(i)
        x = par_subset[tickers[1]]
        y = par_subset[tickers[0]]

        # Adiciona o PERIODO analisado no dataframe
        cubo = cubo.append({'periodo': int(i)}, ignore_index=True)
        # Gera a REGRESSAO e retorna o Coef Ang, os residuos e o z-score. 
        # Retorna tbm o vetor de retorno da regressao para ser usado no ADF
        coef_ang, residuo, zscore, reg  = regressao_multivariada(x, y, i)

        # Teste de Estacionariedade dos Resíduos (ADF)
        adf_stat,adf_sign,adf_pvalue = adftest(residuo, reg)
         
        # Calculo do Fisher
        fisher_r, fisher_pvalue, fisher_lo, fisher_hi = pearsonr_ci(x.diff().fillna(method="bfill"), y.diff().fillna(method="bfill"))

        # Calculo do % Financeiro (C/V)
        pct_fin = pct_financeiro(x,y,coef_ang, float(zscore.tail(1)))

        # Calculo da Meia-vida - ORNSTEIN-UHLENBECK
        meia_vida = calculo_meia_vida1(residuo)

        # Correlação 
        corr = np.corrcoef(x.pct_change().cumsum().dropna(), y.pct_change().cumsum().dropna())[1,0]

        # Se fisher minimo menor que 10%, ou se correlacao < 0, faz adf_sign=0%
        if ((fisher_lo <= 0.1) or (corr<=0) or (coef_ang < 0)):
            adf_sign = "-"
            
        # Preenchimento do cubo
        cubo.loc[ (cubo['periodo']==i), 'preco_y'    ] = float(y.tail(1))
        cubo.loc[ (cubo['periodo']==i), 'preco_x'    ] = float(x.tail(1))
        cubo.loc[ (cubo['periodo']==i), 'ratio'      ] = float(y.tail(1))/float(x.tail(1))
        cubo.loc[ (cubo['periodo']==i), 'coef_ang'   ] = coef_ang 
        cubo.loc[ (cubo['periodo']==i), 'desvio'     ] = float(zscore.tail(1))
        cubo.loc[ (cubo['periodo']==i), 'adf_stat'   ] = adf_stat
        cubo.loc[ (cubo['periodo']==i), 'adf_sign'   ] = adf_sign 
        cubo.loc[ (cubo['periodo']==i), 'pct_fin'    ] = pct_fin
        cubo.loc[ (cubo['periodo']==i), 'meia_vida'  ] = meia_vida
        cubo.loc[ (cubo['periodo']==i), 'corr'       ] = corr
        cubo.loc[ (cubo['periodo']==i), 'fx_corr'    ] = math.ceil(corr*5) # Faixas:  0 a 0.2 / 0.2 a 0.4 / 0.4 a 0.6 / 0.6 a 0.8 / 0.8 a 1.0 ) 
        cubo.loc[ (cubo['periodo']==i), 'dp_residuo' ] = np.std(residuo) 


    ######### Criando algumas variaveis para o backtest
    
    # PER_COINT - Conta quantos periodos estao cointegrados
    cubo_cointegrado = cubo.loc[cubo['adf_sign'] != '-']  
    periodos_coint = len(cubo_cointegrado)                

    # DESVIO_BETA - desvio do coef angular considerando todos os periodos
    desvio_beta = cubo['coef_ang'].std()/cubo['coef_ang'].mean()    
    
    # DESVIO_BETA1 - desvio do coef angular considerando apenas os periodos cointegrados
    desvio_beta1 = cubo_cointegrado['coef_ang'].std()/cubo_cointegrado['coef_ang'].mean() 
    
    # FX_DESVIO_BETA - cria faixas de desvio_beta - até 0.05; entre 0.05 e 0.10; entre 0.10 e 0.20; acima de 0.20
    if   (desvio_beta <= 0.05): fx_desvio_beta = '<= 0.05'
    elif (desvio_beta <= 0.10): fx_desvio_beta = '<= 0.10'
    elif (desvio_beta <= 0.20): fx_desvio_beta = '<= 0.20'
    else: fx_desvio_beta = '> 0.20'
    
    # FX_DESVIO_BETA1 - cria faixas de desvio_beta - até 0.05; entre 0.05 e 0.10; entre 0.10 e 0.20; acima de 0.20
    if   (desvio_beta1 <= 0.05): fx_desvio_beta1 = '<= 0.05'
    elif (desvio_beta1 <= 0.10): fx_desvio_beta1 = '<= 0.10'
    elif (desvio_beta1 <= 0.20): fx_desvio_beta1 = '<= 0.20'
    else: fx_desvio_beta1 = '> 0.20'
    
    # ADFS99 (True/False) - se todos os periodos do cubo tiverem adf=99%, então adf99=1, senao adf99=0
    adf99_periodos = len(cubo.loc[cubo['adf_sign'] == '99%'])  
    
    cubo['per_coint']             = periodos_coint
    cubo['adf99_periodos']        = adf99_periodos
    cubo['vol_beta_vertical']     = desvio_beta
    cubo['fx_vol_beta_vertical']  = fx_desvio_beta
    cubo['vol_beta_vertical1']    = desvio_beta1
    cubo['fx_vol_beta_vertical1'] = fx_desvio_beta1
    
    return cubo

In [ ]:
def geraCubodecubos(tickers, dtinicio, dtfim):

    start = time.time()

    #### CUBO DE FECHAMENTOS
    cubo_fechamentos = pd.DataFrame(columns=['data','preco_y','preco_x','ratio','periodo', 'adf_stat','adf_sign','coef_ang',
                                             'desvio','pct_fin','meia_vida', 'corr', 'fx_corr','per_coint','vol_beta_vertical','fx_vol_beta_vertical',
                                             'vol_beta_vertical1','fx_vol_beta_vertical1','adf99_periodos','dp_residuo'])

    datas = Get_data(tickers, dtinicio, dtfim,'1d').reset_index()['Date']

    for dat in datas:
        tmp = Get_Cubo(tickers,str(dat),False)
        tmp['data']=str(dat)+'-19-00'   # Adicionando 19h00 para o cubo de fechamento
        cubo_fechamentos = cubo_fechamentos.append(tmp)


    #### CUBO INTRADAY
    cubo_intraday    = pd.DataFrame(columns=['data','preco_y','preco_x','ratio','periodo', 'adf_stat','adf_sign','coef_ang',
                                             'desvio','pct_fin','meia_vida', 'corr', 'fx_corr','per_coint','vol_beta_vertical','fx_vol_beta_vertical',
                                             'vol_beta_vertical1','fx_vol_beta_vertical1','adf99_periodos','dp_residuo'])

    datahora = Get_data(tickers,str(dtinicio+'-10-00') , str(dtfim+'-17-00'),'15m').reset_index()['Date']

    for dthora in datahora:

        # Convertendo 2018-01-02 10:30 para 2018-01-02-10-30
        dthora1 = dthora.split(' ')
        dthora2 = dthora1[0]+"-"+dthora1[1]
        dthora1 = dthora2.split(':')
        dthora2 = dthora1[0]+"-"+dthora1[1]
        tmp = Get_Cubo(tickers,str(dthora2),True)
        tmp['data']=str(dthora2)
        cubo_intraday = cubo_intraday.append(tmp)


    ### Concatenando e ordenando os dois cubos
    cubo_total = cubo_fechamentos.append(cubo_intraday)
    cubo_total = cubo_total.sort_values(['data','periodo'])
    cubo_total.reset_index(drop=True, inplace=True)

    #display(cubo_total)

    end = time.time()
    print('Tempo total = {total}'.format(total = (end - start)))
    
    return cubo_total

In [10]:
tickers = ['ITUB4','GOLL4']

cubototal = geraCubodecubos(tickers, '2020-04-01','2020-06-30')

cubototal.to_excel('cubo_ITUB4_GOLL4.xlsx')

display(cubototal)


Tempo total = 1006.532053232193


,data,preco_y,preco_x,ratio,periodo,adf_stat,adf_sign,coef_ang,desvio,pct_fin,meia_vida,corr,fx_corr,per_coint,vol_beta_vertical,fx_vol_beta_vertical,vol_beta_vertical1,fx_vol_beta_vertical1,adf99_periodos,dp_residuo
0,2020-04-01-10-15,22.14,10.13,2.19,100.00,-2.95,-,0.31,-1.53,694%,10.00,0.93,5.00,2,0.07,<= 0.10,0.01,<= 0.05,0,1.09
1,2020-04-01-10-15,22.14,10.13,2.19,120.00,-2.95,-,0.36,-1.51,615%,14.00,0.93,5.00,2,0.07,<= 0.10,0.01,<= 0.05,0,1.20
2,2020-04-01-10-15,22.14,10.13,2.19,140.00,-3.07,-,0.36,-1.59,603%,14.00,0.93,5.00,2,0.07,<= 0.10,0.01,<= 0.05,0,1.14
3,2020-04-01-10-15,22.14,10.13,2.19,160.00,-3.28,-,0.38,-1.63,574%,15.00,0.92,5.00,2,0.07,<= 0.10,0.01,<= 0.05,0,1.17
4,2020-04-01-10-15,22.14,10.13,2.19,180.00,-3.03,-,0.38,-1.80,581%,18.00,0.91,5.00,2,0.07,<= 0.10,0.01,<= 0.05,0,1.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15844,2020-06-30-19-00,25.42,18.55,1.37,180.00,-3.88,95%,0.40,-0.14,340%,12.00,0.89,5.00,9,0.04,<= 0.05,0.04,<= 0.05,0,1.19
15845,2020-06-30-19-00,25.42,18.55,1.37,200.00,-4.06,95%,0.41,-0.20,338%,12.00,0.90,5.00,9,0.04,<= 0.05,0.04,<= 0.05,0,1.15
15846,2020-06-30-19-00,25.42,18.55,1.37,220.00,-4.24,95%,0.42,-0.51,324%,13.00,0.90,5.00,9,0.04,<= 0.05,0.04,<= 0.05,0,1.18
15847,2020-06-30-19-00,25.42,18.55,1.37,240.00,-3.84,95%,0.43,-0.71,322%,16.00,0.90,5.00,9,0.04,<= 0.05,0.04,<= 0.05,0,1.30


In [25]:
#cubototal = pd.read_excel('cubo_ITUB4_BBSE3.xlsx')
# display(cubototal)

In [11]:
#######################################################
#        CONSTRUCAO DE VARIAVEIS TEMPORAIS            #
#######################################################
# - ADFCOUNT  - Qtd Dias com ADF=99% em cada período  #
# - BETAVOL63 - Vol do beta em janela de 63 dias      #
# - BETAVOL10 - Vol do beta em janela de 10 dias      #
# ** TODAS BASEADAS EM FECH DIARIOS ('19-00')         #
#######################################################

for periodo in [100,120,140,160,180,200,220,240,250]:
    
    adfcount = 0    
    cubo_tmp = cubototal[cubototal['periodo']==periodo]
    cubo_tmp_19h = cubototal[ (cubototal['periodo']==periodo) & (cubototal['data'].apply(lambda x: x[-5:]) == '19-00') ] # Bet Rot
    
    date_unique = cubo_tmp['data'].unique()
    
    for date in date_unique:

        row = cubo_tmp.loc[cubo_tmp['data'] == date]
        horario = row['data'].str[-5:].values[0]
        adf = row['adf_sign'].values[0]
   
        # ADFCOUNT - Se for 99% quando o horario for 19h, soma 1, senao recomeça a contagem
        if (horario == '19-00'):
            if (adf == '99%'): adfcount += 1
            else: adfcount = 0

        # BETAVOL - concatena a serie de fechamentos (19h) com o ponto atual e calcula a volatilidade percentual
        last_rows_19h = cubo_tmp_19h.loc[cubo_tmp_19h['data'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d-%H-%M')) < datetime.strptime(date, '%Y-%m-%d-%H-%M')]   # Beta Rot

        beta63 = pd.concat([last_rows_19h,row]).tail(63) # Beta Rot   ALTERAR PARA 0.2*PERIODO
        beta10 = pd.concat([last_rows_19h,row]).tail(10) # Beta Rot   ALTERAR PARA 0.2*PERIODO
        betavol63 = beta63['coef_ang'].std() / beta63['coef_ang'].mean()   # Beta Rot
        betavol10 = beta10['coef_ang'].std() / beta10['coef_ang'].mean()   # Beta Rot
       
        # Cria vars de faixas dos betavol10 e betavol63 - até 0.05; entre 0.05 e 0.10; entre 0.10 e 0.20; acima de 0.20
        if   (betavol63 <= 0.05): fx_betavol63 = '<= 0.05'
        elif (betavol63 <= 0.10): fx_betavol63 = '<= 0.10'
        elif (betavol63 <= 0.20): fx_betavol63 = '<= 0.20'
        else: fx_betavol63 = '> 0.20'
        
        if   (betavol10 <= 0.05): fx_betavol10 = '<= 0.05'
        elif (betavol10 <= 0.10): fx_betavol10 = '<= 0.10'
        elif (betavol10 <= 0.20): fx_betavol10 = '<= 0.20'
        else: fx_betavol10 = '> 0.20'
        
         
        cubototal.loc[ (cubototal['periodo']==periodo) & (cubototal['data'] == date) , 'adfcount'    ] = adfcount
        cubototal.loc[ (cubototal['periodo']==periodo) & (cubototal['data'] == date) , 'betavol63'   ] = betavol63
        cubototal.loc[ (cubototal['periodo']==periodo) & (cubototal['data'] == date) , 'fx_betavol63'] = fx_betavol63
        cubototal.loc[ (cubototal['periodo']==periodo) & (cubototal['data'] == date) , 'betavol10'   ] = betavol10
        cubototal.loc[ (cubototal['periodo']==periodo) & (cubototal['data'] == date) , 'fx_betavol10'] = fx_betavol10
        


In [14]:
cubototal.to_excel('cubo_ITUB4_GOLL4_1.xlsx')
cubototal_backup = cubototal.copy()
display(cubototal)

In [15]:
def Busca_trade(cubo_of, coint=2, dp=2., adf_sign =0.95, meia_vida = 10, voltando = False):
    
    date_unique = cubo_of['data'].unique()
    a= 0
    
    trade_select    = pd.DataFrame(columns=['data','preco_y','preco_x','ratio','periodo', 'adf_stat','adf_sign','coef_ang',
                                             'desvio','pct_fin','meia_vida', 'corr', 'fx_corr','per_coint','vol_beta_vertical','fx_vol_beta_vertical',
                                             'vol_beta_vertical1','fx_vol_beta_vertical1','adf99_periodos'])

    full_trade_select = pd.DataFrame(columns=['data','preco_y','preco_x','ratio','periodo', 'adf_stat','adf_sign','coef_ang',
                                             'desvio','pct_fin','meia_vida', 'corr', 'fx_corr','per_coint','vol_beta_vertical','fx_vol_beta_vertical',
                                             'vol_beta_vertical1','fx_vol_beta_vertical1','adf99_periodos'])
    
    #arredondando o desvio
    cubo_of['desvio'] = cubo_of['desvio'].apply(lambda x: round(x,2))
    #criando o desvio com valor absoluto
    cubo_of['desvio_abs'] = cubo_of['desvio'].apply(lambda x: abs(x))
    #removendo elementos nao cointegrados adf_sign = '-'
    cubo_of_index = cubo_of[ cubo_of['adf_sign'] == '-' ].index
    cubo_of.drop(cubo_of_index , inplace=True)
    #removendo as linhas de close diario com hora = '19:00'
    cubo_of_index = cubo_of[ cubo_of['data'].apply(lambda x: x[-5:]) == '19-00' ].index
    cubo_of.drop(cubo_of_index , inplace=True)
    #definindo a data quarenta como a primeira data pra iniciar o loop
    dt_quarentena = datetime.strptime(date_unique[0], '%Y-%m-%d-%H-%M')

    for date in date_unique:
        
        cubo_var = cubo_of.loc[cubo_of['data'] == date].copy()
        #selecionando linhas com padroes de DP e #perdiodos cointegrados
        var_trade = cubo_var.loc[(cubo_var['desvio_abs'] >= dp) & (cubo_var['per_coint'] >= coint)]
        # Ordenando por t-stat do ADF e pegando o maior valor
        var_trade = var_trade.sort_values(by='adf_stat')
        
        
        #append do trade de cada iteracao, caso haja
        if len (var_trade) != 0:
            
            #pegando a data do trade e criando o dt_obj da mesma , ESTE SERA O TRADE SELECIONADO
            data_trade = var_trade.iloc[0]['data']
            d_obj_trade = datetime.strptime(data_trade, '%Y-%m-%d-%H-%M') 
            
            #print ('Tentativa de adicionar trade da data    => {data}'.format(data = d_obj_trade)) # remover dps do debug
            #print ('Trades em quarentena ate a data:        => {data}'.format(data = dt_quarentena))# remover dps do debug
            full_trade_select = full_trade_select.append(var_trade.iloc[0]) # remover dps do debug
            
            #verificando se a data do trade e maior que a data de quarentena de 1/4 de meia vida do trade anterior do par
            if (d_obj_trade >= dt_quarentena):
                #print('***************************************')
                #print('********** Trade Adicionado ***********')
                #print('***************************************')
                trade_select = trade_select.append(var_trade.iloc[0])
                #criando o periodo de quarentana arredondado para cima
                dias_quarentena = int(math.ceil(var_trade.iloc[0]['meia_vida'] / 4))
                #criando a date_obj da quarenta a ser reseitada em data util B3
                dt_quarentena = datetime.strptime(ajusta_B3Date(data_trade[:-6], dias_quarentena), '%Y-%m-%d-%H-%M')
            
    #print ('\n')
    
    print ('TOTAL DE TRADES IDENTIFICADOS NO PERIODO  => {data}'.format(data = len(full_trade_select))) # remover dps do debug
    print ('TOTAL DE TRADES SELECIONADOS NO PERIODO   => {data}'.format(data = len(trade_select))) # remover dps do debug
    
    #display(full_trade_select)
    display(trade_select)
    full_trade_select.to_excel('Total_trades.xlsx') # remover dps do debug
    
    return trade_select

In [16]:
trades = Busca_trade(cubototal_backup)

trades.to_excel('Trades_adicionados_ITUB_GOLL.xlsx')

TOTAL DE TRADES IDENTIFICADOS NO PERIODO  => 116
TOTAL DE TRADES SELECIONADOS NO PERIODO   => 3


,data,preco_y,preco_x,ratio,periodo,adf_stat,adf_sign,coef_ang,desvio,pct_fin,...,per_coint,vol_beta_vertical,fx_vol_beta_vertical,vol_beta_vertical1,fx_vol_beta_vertical1,adf99_periodos,adfcount,beta_mean,desvio_abs,dp_residuo
8,2020-04-01-10-15,22.14,10.13,2.19,250.00,-3.35,90%,0.32,-2.25,675%,...,2,0.07,<= 0.10,0.01,<= 0.05,0,0.00,0.32,2.25,1.20
10910,2020-06-03-10-00,26.57,16.51,1.61,140.00,-3.30,90%,0.34,2.49,21%,...,7,0.10,<= 0.10,0.07,<= 0.10,0,0.00,0.34,2.49,1.10
13557,2020-06-18-11-30,27.70,19.02,1.46,160.00,-3.51,90%,0.37,2.23,26%,...,6,0.05,<= 0.10,0.05,<= 0.05,0,0.00,0.37,2.23,1.12


In [31]:
#removendo as linhas de close diario com hora = '19:00'
cubototal_index = cubototal[ cubototal['data'].apply(lambda x: x[-5:]) == '19-00' ].index
cubototal.drop(cubototal_index , inplace=True)
cubototal['date_obj'] = cubototal['data'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d-%H-%M')) 
#par_periodo = cubototal ['data','preco_y', 'preco_x', 'ratio', 'desvio']
#display (cubototal)

def Simula_Trade (trade_select, cubototal, dp, debug = False):
    
    #iteracao dentro dos trades selecionados
    for index, trade in trade_select.iterrows():
        
        if debug:
            print ('****************************************************************')
            print ('******************INICIO DE ANALISE DO TRADE********************')
            print ('****************************************************************\n')
        
        #criacao das datas limites para percorrer o trade
        dt_trade = datetime.strptime(trade['data'], '%Y-%m-%d-%H-%M')
        dt_meiavida = datetime.strptime(ajusta_B3Date(trade['data'][:-6], int(trade['meia_vida'])), '%Y-%m-%d-%H-%M')
        periodo_trade = trade['periodo'] #periodo do trade
        #selecionando o periodo do cubo entre as datas limites e o periodo correspondente ao trade selecionado
        cubo_slice = cubototal.loc[(cubototal['date_obj'] > dt_trade) & (cubototal['date_obj'] <= dt_meiavida) & (cubototal['periodo'] == periodo_trade)]
        #criando as variaveis do trade
        cot_x = trade['preco_x']
        cot_y = trade['preco_y']
        ratio_trade = trade['ratio']
        dp_residuo = trade['dp_residuo']*dp
        beta = trade['coef_ang']
        target_fin = dp_residuo
        
        #definindo as ratios conforme o trade tenha sido feito no desvio negativo ou positivo
        if trade_select.loc[index,'desvio'] > 0:
            desvio_positivo = True
            ratio_gain = (cot_y - dp_residuo)/cot_x
            ratio_loss = (cot_y + dp_residuo)/cot_x
            fin_trade = cot_y - (cot_x*beta)
        else:
            desvio_positivo = False
            ratio_gain = (cot_y + dp_residuo)/cot_x
            ratio_loss = (cot_y - dp_residuo)/cot_x
            fin_trade = (cot_x*beta) - cot_y
        
        #definindo variaveis controle que trade nao foi encontrado
        fin_status = False
        ratio_status = False
        passou_aqui = False
        
        #iterando o trade em questao dentro do periodo da meia-vida
        for var_index , trade_iterado in cubo_slice.iterrows():
            
            cot_x_atual = trade_iterado['preco_x']
            cot_y_atual = trade_iterado['preco_y']
            data_atual = datetime.strptime(trade_iterado['data'], '%Y-%m-%d-%H-%M')
        
            if desvio_positivo:
                ratio_atual = (cot_y_atual)/cot_x_atual
                fin_atual =  (cot_x_atual*beta) - cot_y_atual
            else:
                ratio_atual = (cot_y_atual)/cot_x_atual
                fin_atual = cot_y_atual - (cot_x_atual*beta)
            
            #calculando o financeiro do trade no momento atual
            fin_acumulado =  fin_atual + fin_trade
            
            #verificando se o financeiro excedeu o target e se sera pela primeira vez (Fin_status == False)
            if (fin_acumulado > target_fin) and (fin_status == False):                
                result_fin = 1
                fin_status = True 
            elif (fin_acumulado < (target_fin*-1)) and (fin_status == False):
                result_fin = 0
                fin_status = True
            
            #caso tenha encontrado o financeiro final do trade, adiciona a base de trades, nao sera sobrescrito pois fin_status== True
            if fin_status and passou_aqui == False:
                fin_final_fin = fin_acumulado
                dt_close_fin = data_atual
                days_elapsed = dt_close_fin - dt_trade
                trade_select.loc[index,'elapsed_days'] = days_elapsed.days 
                trade_select.loc[index,'result_fin'] = fin_final_fin
                trade_select.loc[index,'dt_close_fin'] = dt_close_fin
                passou_aqui = True # soh eh necessario para debug
            
            if debug:
                
                print('Data do trade => {date}'.format (date=dt_trade))
                print('Data ATUAL do trade => {date}'.format (date=data_atual))
                print('-------------------------')
                print('Ratio inicial => {ratio}'.format(ratio=ratio_trade))            
                print('Ratio atual => {ratio}'.format(ratio=ratio_atual))
                print('Ratio Target Loss => {ratio}'.format(ratio=ratio_loss))
                print('Ratio Target Gain => {ratio}'.format(ratio=ratio_gain))            
                print('-------------------------')
                print('Cotacao inicial X =>{cot}'.format(cot=cot_x))
                print('Cotacao inicial Y =>{cot}'.format(cot=cot_y))
                print('Cotacao atual X =>{cot}'.format(cot=cot_x_atual))
                print('Cotacao atual Y =>{cot}'.format(cot=cot_y_atual))
                print('-------------------------')            
                print('Financeiro inicial => {fin}'.format(fin=fin_trade))
                print('Financeiro atual => {fin}'.format(fin=fin_atual))
                print('Financeiro acumulado => {fin}'.format(fin=fin_atual+fin_trade))
                print('Target Financeiro => {fin}'.format(fin=target_fin))
                print('-------------------------')   
                print('Status Financeiro => {fin}'.format(fin=fin_status))
                if fin_status:
                    print('Financeiro Final        [SAIDA FINANCEIRA] => {fin}'.format(fin=fin_final_fin))
                    print('Data Encerramento Trade [SAIDA FINANCEIRA] => {fin}'.format(fin=dt_close_fin))
                print('\n')
                print('\n')
        
        #fim do for de procura do trade a meia vida
        if fin_status == False:
            trade_select.loc[index,'result_fin'] = fin_acumulado
            trade_select.loc[index,'dt_close_fin'] = data_atual
            days_elapsed = data_atual - dt_trade
            trade_select.loc[index,'elapsed_days'] = days_elapsed.days
            #trade_select.loc[index,'result_fin']
            
            #trade_select.loc[index,'result_ratio'] = index 
    return trade_select

In [32]:
trades_atualizado = Simula_Trade(trades, cubototal, 1 , False) # Ultimo parametro (debug) como True para imprimir etapas

In [33]:
trades_atualizado

,data,preco_y,preco_x,ratio,periodo,adf_stat,adf_sign,coef_ang,desvio,pct_fin,...,vol_beta_vertical,fx_vol_beta_vertical,vol_beta_vertical1,fx_vol_beta_vertical1,adf99_periodos,desvio_abs,dp_residuo,elapsed_days,result_fin,dt_close_fin
8,2020-04-01-10-15,22.14,10.13,2.19,250.00,-3.35,90%,0.32,-2.25,675%,...,0.07,<= 0.10,0.01,<= 0.05,0,2.25,1.20,6.00,1.26,2020-04-07 10:15:00
10910,2020-06-03-10-00,26.57,16.51,1.61,140.00,-3.30,90%,0.34,2.49,21%,...,0.10,<= 0.10,0.07,<= 0.10,0,2.49,1.10,1.00,1.22,2020-06-04 10:00:00
13557,2020-06-18-11-30,27.70,19.02,1.46,160.00,-3.51,90%,0.37,2.23,26%,...,0.05,<= 0.10,0.05,<= 0.05,0,2.23,1.12,5.00,1.22,2020-06-23 16:00:00
20610,2020-07-27-12-00,27.67,18.57,1.49,100.00,-3.90,95%,0.58,2.19,39%,...,0.13,<= 0.20,0.13,<= 0.20,1,2.19,0.96,4.00,0.42,2020-07-31 16:45:00
21321,2020-07-30-10-15,28.10,18.52,1.52,100.00,-3.53,90%,0.54,2.17,35%,...,0.09,<= 0.10,0.09,<= 0.10,1,2.17,1.03,1.00,0.90,2020-07-31 16:45:00


In [35]:
trades_atualizado.to_excel('Trades_closed-ITUB-GOLL.xlsx')